In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tqdm
import os
import glob
import cv2

## Dataloader

In [2]:
meta = pd.read_csv("/content/drive/My Drive/chestxray14/original_split/original_split_train.csv")
data_entry = pd.read_csv("/content/drive/My Drive/chestxray14/Data_Entry_2017.csv")

In [3]:
len(meta)

78484

In [ ]:
meta.head()

,Image Index,Atelectasis,Cardiomegaly,Effusion,Infiltration,Mass,Nodule,Pneumonia,Pneumothorax,Consolidation,Edema,Emphysema,Fibrosis,Pleural_Thickening,Hernia,path,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11
0,00000003_000.png,0,0,0,0,0,0,0,0,0,0,0,0,0,1,/content/drive/My Drive/chestxray14/images512/...,Hernia,0,3,81,F,PA,2582,2991,0.143,0.143,NaN
1,00000003_001.png,0,0,0,0,0,0,0,0,0,0,0,0,0,1,/content/drive/My Drive/chestxray14/images512/...,Hernia,1,3,74,F,PA,2500,2048,0.168,0.168,NaN
2,00000003_002.png,0,0,0,0,0,0,0,0,0,0,0,0,0,1,/content/drive/My Drive/chestxray14/images512/...,Hernia,2,3,75,F,PA,2048,2500,0.168,0.168,NaN
3,00000003_003.png,0,0,0,1,0,0,0,0,0,0,0,0,0,1,/content/drive/My Drive/chestxray14/images512/...,Hernia|Infiltration,3,3,76,F,PA,2698,2991,0.143,0.143,NaN
4,00000003_004.png,0,0,0,0,0,0,0,0,0,0,0,0,0,1,/content/drive/My Drive/chestxray14/images512/...,Hernia,4,3,77,F,PA,2500,2048,0.168,0.168,NaN


In [ ]:
data_entry.head()

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,0.143,NaN
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143,NaN
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168,NaN
3,00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,0.171,NaN
4,00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143,0.143,NaN


In [ ]:
def generate_path(row):
  return '/content/drive/My Drive/chestxray14/images512/{}'.format(
      row['Image Index']
  )

In [ ]:
meta['path'] = meta.apply(lambda x : generate_path(x), axis=1)

In [ ]:
meta = pd.merge(meta, data_entry, on='Image Index')

In [ ]:
meta = meta[meta['View Position'] == 'AP']

In [ ]:
meta_labels = meta.iloc[:, 1:15]

In [ ]:
data_ds = tf.data.Dataset.from_tensor_slices((meta['path'],np.array(meta_labels), meta['Image Index']))

In [ ]:
def decode_img(img):
  img = tf.image.decode_png(img, channels=3)
  img = tf.image.convert_image_dtype(img, tf.float32)
  return tf.image.resize(img, (224,224))

def process_path(path, label, sid):
  img = tf.io.read_file(path)
  img = decode_img(img)
  return img, label, sid

In [ ]:
data_im = data_ds.map(process_path, num_parallel_calls=tf.data.experimental.AUTOTUNE)\
    .apply(tf.data.experimental.ignore_errors()).cache('/content/train_cache')

### Create TFRecord

In [ ]:
len(meta)

2824

In [ ]:
# TR_SHARDS = 10
TR_SHARDS = 3
GCS_OUTPUT = '/content/drive/Shared drives/CMB - corpora/Chest_x-ray_report_Jan2020/Chest Xray14 Tfrecord/val_ap/' 

In [ ]:
import math

data_size = len(meta)
shard_size = math.ceil(1.0 * data_size / TR_SHARDS)

In [ ]:
shard_size

942

In [ ]:
def recompress_image(image, label, sid):
  image = image * 255
  image = tf.cast(image, tf.uint8)
  image = tf.image.encode_jpeg(image, optimize_size=True, chroma_downsampling=False)
  return image, label, sid

In [ ]:
data_imc = data_im.map(recompress_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
data_imc = data_imc.batch(shard_size).prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
# Three types of data can be stored in TFRecords: bytestrings, integers and floats
# They are always stored as lists, a single data element will be a list of size 1

def _bytestring_feature(list_of_bytestrings):
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=list_of_bytestrings))

def _int_feature(list_of_ints): # int64
  return tf.train.Feature(int64_list=tf.train.Int64List(value=list_of_ints))

def _float_feature(list_of_floats): # float32
  return tf.train.Feature(float_list=tf.train.FloatList(value=list_of_floats))
  

def to_tfrecord(tfrec_filewriter, img, label, sid):  
  
  feature = {
      "image": _bytestring_feature([img]), 
      "label":     _float_feature(label.reshape(-1)), 
      "image_index": _bytestring_feature([sid])
    }
  return tf.train.Example(features=tf.train.Features(feature=feature))


In [ ]:
print("Writing TFRecords")
for shard, (image, label, sid) in tqdm.tqdm(enumerate(data_imc)):
  # batch size used as shard size here
  shard_size = image.numpy().shape[0]
  # good practice to have the number of records in the filename
  filename = GCS_OUTPUT + "{:02d}-{}.tfrec".format(shard, shard_size)
  if os.path.exists(filename): 
    continue
  
  with tf.io.TFRecordWriter(filename) as out_file:
    for i in tqdm.tqdm(range(shard_size)):
      example = to_tfrecord(out_file,
                            image.numpy()[i],
                            label.numpy()[i],
                            sid.numpy()[i])
      out_file.write(example.SerializeToString())
    print("Wrote file {} containing {} records".format(filename, shard_size))

0it [00:00, ?it/s]

Writing TFRecords



100%|██████████| 942/942 [00:11<00:00, 85.62it/s]
1it [35:19, 2119.17s/it]

Wrote file /content/drive/Shared drives/CMB - corpora/Chest_x-ray_report_Jan2020/Chest Xray14 Tfrecord/val_ap/00-942.tfrec containing 942 records



100%|██████████| 942/942 [00:11<00:00, 84.39it/s]
2it [39:20, 1555.88s/it]

Wrote file /content/drive/Shared drives/CMB - corpora/Chest_x-ray_report_Jan2020/Chest Xray14 Tfrecord/val_ap/01-942.tfrec containing 942 records



100%|██████████| 910/910 [00:11<00:00, 81.38it/s]


Wrote file /content/drive/Shared drives/CMB - corpora/Chest_x-ray_report_Jan2020/Chest Xray14 Tfrecord/val_ap/02-910.tfrec containing 910 records


3it [43:11, 863.77s/it] 


In [ ]:
def read_tfrecord(example):
    features = {
        "image": tf.io.FixedLenFeature([], tf.string),  # tf.string = bytestring (not text string)
        "label": tf.io.VarLenFeature(tf.float32)        # a certain number of floats0
        "size":  tf.io.FixedLenFeature([2], tf.int64),  # two integers
    }
    # decode the TFRecord
    example = tf.io.parse_single_example(example, features)
    
    # FixedLenFeature fields are now ready to use: exmple['size']
    # VarLenFeature fields require additional sparse_to_dense decoding
    
    image = tf.image.decode_jpeg(example['image'], channels=3)
    label  = tf.reshape(tf.sparse.to_dense(example['label']), [-1, 14])
    height = example['size'][0]
    width  = example['size'][1]
    image = tf.reshape(image, [height, width, 3])
    return image, label

In [ ]:
# read from TFRecords. For optimal performance, read from multiple
# TFRecord files at once and set the option experimental_deterministic = False
# to allow order-altering optimizations.

option_no_order = tf.data.Options()
option_no_order.experimental_deterministic = False

filenames = tf.io.gfile.glob(GCS_OUTPUT + "*.tfrec")
train_dsr = tf.data.TFRecordDataset(filenames, num_parallel_reads=tf.data.experimental.AUTOTUNE)
train_dsr = train_dsr.with_options(option_no_order)
train_dsr = train_dsr.map(read_tfrecord, num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_dsr = train_dsr.shuffle(300)